In [37]:
# Import the required libraries.
import re
import math
import random
import collections
import operator
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import precision_recall_fscore_support, f1_score, accuracy_score
from sklearn.metrics import confusion_matrix
from collections import defaultdict, Counter

from keras.utils import to_categorical
from keras.layers import *
from keras.models import Model
from keras import Model, Sequential
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import *
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight

import math

random.seed(11)
np.random.seed(11)

In [2]:
!ls

NER-Dataset-10Types-Train.txt
NER-Dataset--TestSet.txt
NER-Dataset-Train.txt
Q1 - NER Prediction 10 Types (buggy).ipynb
Q1 - NER prediction-10 Types (no test output)-Copy1.ipynb
Q1 - NER prediction-10 Types (no test output).ipynb
Q1 - NER prediction-10 Types (with test output).ipynb
Q1 - NER prediction.ipynb
Q1 - NER prediction (with test output).ipynb
Q2.ipynb
Q2 - NER prediction-10 Types.ipynb


In [3]:
with open('NER-Dataset-10Types-Train.txt', 'r') as f:
    ner_dataset = f.readlines()

In [4]:
sentences = []
words = []
tags = []
for line in ner_dataset:
    line = line.strip()
    if line == '':
        sentences.append((words, tags))
        words = []
        tags = []
    else:
        word, tag = line.split('\t')
        words.append(word)
        tags.append(tag)

if len(words) > 0:
    sentences.append((words, tags))
    words = []
    tags= []

In [5]:
len(sentences)

900

In [6]:
vocab_counts = Counter(sum([a[0] for a in sentences], [])).most_common()

In [7]:
words_to_keep = set([word for word, count in vocab_counts if count > 1])
len(words_to_keep)

1456

In [8]:
# parsed_sentences = [([w if w in words_to_keep else 'UNK' for w in words], tags) for words, tags in sentences]

In [9]:
with open('NER-Dataset--TestSet.txt', 'r') as f:
    test_dataset = f.readlines()

test_sentences = []
words = []
for line in test_dataset:
    line = line.strip()
    if line == '':
        test_sentences.append((words,))
        words = []
    else:
        word = line
        words.append(word)

if len(words) > 0:
    test_sentences.append((words,))
    words = []


In [10]:
len(sentences), len(test_sentences)

(900, 100)

In [11]:
word_features = ['twoDigitNum',
                'fourDigitNum',
                'containsDigitAndAlpha',
                'containsDigitAndDash',
                'containsDigitAndSlash',
                'containsDigitAndComma',
                'containsDigitAndPeriod',
                'otherNum',
                'allCaps',
                'capPeriod',
                'firstWord',
                'initCap',
                'lowerCase',
                'other']

In [12]:
def get_word_features(sentence):
    features = []
    ## Optimize and use an Enum!
    firstword = True
    for word in sentence:
        if word.isnumeric() and len(word) == 2:
            features.append('twoDigitNum')
        elif word.isnumeric() and len(word) == 4:
            features.append('fourDigitNum')
        elif word.isalnum() and not word.isalpha() and not word.isnumeric():
            features.append('containsDigitAndAlpha')
        elif word.replace('-', '').isnumeric():
            features.append('containsDigitAndAlpha')
        elif word.replace('/', '').isnumeric():
            features.append('containsDigitAndSlash')
        elif word.replace('.', '').replace(',', '').isnumeric() and ',' in word:
            features.append('containsDigitAndComma')
        elif word.replace('.', '').isnumeric():
            features.append('containsDigitAndPeriod')
        elif word.isnumeric():
            features.append('otherNum')
        elif word.isupper():
            features.append('allCaps')
        elif len(word) == 2 and word[0].isupper() and word[1] == '.':
            features.append('capPeriod')
        elif firstword:
            features.append('firstWord')
        elif word[0].isupper():
            features.append('initCap')
        elif word.islower():
            features.append('lowerCase')
        else:
            features.append('other')
        firstword = False

    return features

In [13]:
max_len_found = max(len(s[0]) for s in sentences)

In [14]:
max_len = max_len_found + ((50 - (max_len_found % 50)) % 50)

In [15]:
eye_mat = list(np.eye(len(word_features)))
wordfeat2float = {feat: eye_mat[i] for i, feat in enumerate(word_features)}

In [16]:
word2idx = {'UNK': 0, 'PAD': 1}
word2idx.update({word: i + 2 for i, word in enumerate(sorted(words_to_keep))})

In [17]:
def numberize_sentence(words, max_len=50):
    features = get_word_features(words)
    word_idx = [word2idx[w] if w in word2idx.keys() else word2idx['UNK'] for w in words]
    feat_np = [wordfeat2float[f] for f in features]
    word_padding = [word2idx['PAD'] for _ in range(max_len - len(word_idx))]
    feat_padding = [np.ones((len(word_features),)) * 2 for _ in range(max_len - len(word_idx))]
    word_idx = np.asarray(word_idx + word_padding)
    feat_np = np.asarray(feat_np + feat_padding)
    return word_idx, feat_np

In [18]:
labels = set.union(*(set(s[1]) for s in sentences))
n_labels = len(labels)
eye_mat = list(np.eye(len(labels)))
labels2float = {feat: eye_mat[i] for i, feat in enumerate(labels)}

def numberize_labels(gt_labels, max_len=50):
    labels_np = [labels2float[l] for l in gt_labels]
    labels_padding = [labels2float['O'] for _ in range(max_len - len(gt_labels))]
    return np.asarray(labels_np + labels_padding)

In [19]:
def create_model():
    input_words = Input(shape=(max_len,))
    input_feats = Input(shape=(max_len, len(word_features)))
    masked_words = Masking(mask_value = 1)(input_words)
    masked_feats = Masking(mask_value = 2)(input_feats)
    emb = Embedding(input_dim=(len(word2idx)), output_dim=50, input_length=max_len)(masked_words)
    drop_emb = Dropout(0.1)(emb)
    concat_out = Concatenate()([drop_emb, masked_feats])
    rnn_out = Bidirectional(SimpleRNN(units=100, return_sequences=True, recurrent_dropout=0.1))(concat_out)
    dense_out = TimeDistributed(Dense(n_labels, activation="softmax"))(rnn_out)
    model = Model(inputs=[input_words, input_feats], outputs=dense_out)
    model.summary()
    return model

In [20]:
parsed_sentences = [(numberize_sentence(s[0]), numberize_labels(s[1])) for s in sentences]

In [21]:
parsed_test_sentences = [numberize_sentence(s[0]) for s in test_sentences]

In [22]:
Counter(sum([s[1] for s in sentences], []))

Counter({'O': 16508,
         'B-musicartist': 22,
         'I-musicartist': 21,
         'B-company': 71,
         'B-geo-loc': 114,
         'B-product': 32,
         'B-person': 176,
         'B-tvshow': 11,
         'I-tvshow': 8,
         'B-other': 89,
         'I-other': 132,
         'B-facility': 41,
         'I-facility': 38,
         'I-person': 91,
         'B-movie': 12,
         'I-movie': 17,
         'I-geo-loc': 26,
         'B-sportsteam': 14,
         'I-product': 33,
         'I-sportsteam': 5,
         'I-company': 19})

In [23]:
Counter(sum([np.argmax(s[1], axis=-1).tolist() for s in parsed_sentences], []))

Counter({15: 44028,
         14: 22,
         19: 21,
         3: 71,
         13: 114,
         1: 32,
         7: 176,
         16: 11,
         5: 8,
         11: 89,
         17: 132,
         20: 41,
         2: 38,
         9: 91,
         12: 12,
         18: 17,
         4: 26,
         6: 14,
         10: 33,
         8: 5,
         0: 19})

In [41]:
# class_weights = compute_class_weight('balanced', np.unique(sum([np.argmax(s[1], axis=-1).tolist() for s in parsed_sentences], [])), sum([np.argmax(s[1], axis=-1).tolist() for s in parsed_sentences], []))

In [42]:
# class_weights

array([1.12781955e+02, 6.69642857e+01, 5.63909774e+01, 3.01810865e+01,
       8.24175824e+01, 2.67857143e+02, 1.53061224e+02, 1.21753247e+01,
       4.28571429e+02, 2.35478807e+01, 6.49350649e+01, 2.40770465e+01,
       1.78571429e+02, 1.87969925e+01, 9.74025974e+01, 4.86703267e-02,
       1.94805195e+02, 1.62337662e+01, 1.26050420e+02, 1.02040816e+02,
       5.22648084e+01])

In [44]:
# Build the test and training sets of sentences.
kf = KFold(n_splits = 5, shuffle = False)
parsed_sentences = np.asarray(parsed_sentences)
scores = []
scores1 = []
y_pred_idx = []
y_pred_idx1 = []
y_test_idx = []
y_test_idx1 = []

preds = []

for train_index, test_index in kf.split(parsed_sentences):
    train_data = parsed_sentences[train_index]
    test_data = parsed_sentences[test_index]
    X_train = [np.asarray([a[0][0] for a in train_data]), np.asarray([a[0][1] for a in train_data])]
    Y_train = np.asarray([a[1] for a in train_data])
    X_test = [np.asarray([a[0][0] for a in test_data]), np.asarray([a[0][1] for a in test_data])]
    Y_test = np.asarray([a[1] for a in test_data])
    model = create_model()
    model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    model.fit(X_train, Y_train, epochs=3, validation_split=0.1, batch_size=4)

    y_pred_padded = np.argmax(model.predict(X_test), axis=-1)
    y_true_padded = np.argmax(Y_test, axis=-1)
    
    for i in range(X_test[0].shape[0]):
        for j in range(X_test[0].shape[1]):
            if X_test[0][i][j] == word2idx['PAD']:
                continue
            else:
                pred = y_pred_padded[i][j]
                true = y_true_padded[i][j]
                y_pred_idx.append(pred)
                y_test_idx.append(true)
                scores.append(pred == true)

    prec_, rec_, fscore_, _ = precision_recall_fscore_support(y_test_idx, y_pred_idx, average = 'macro')
    print('[still updating...] Accuracy: {}, Precision: {}, Recall: {}, FScore: {}'.format(np.asarray(scores).mean(), prec_, rec_, fscore_))
    
prec, rec, fscore, _ = precision_recall_fscore_support(y_test_idx, y_pred_idx, average = 'macro')
print('Accuracy: {}, Precision: {}, Recall: {}, FScore: {}'.format(np.asarray(scores).mean(), prec, rec, fscore))

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
masking_23 (Masking)            (None, 50)           0           input_23[0][0]                   
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 50, 50)       72900       masking_23[0][0]                 
__________________________________________________________________________________________________
input_24 (InputLayer)           (None, 50, 14)       0                                            
___________________________________________________________________________________________

/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 648 samples, validate on 72 samples
Epoch 1/3
648/648 [==============================] - 31s 49ms/step - loss: 0.1598 - accuracy: 0.9369 - val_loss: 0.1139 - val_accuracy: 0.9510
Epoch 2/3
648/648 [==============================] - 28s 43ms/step - loss: 0.1105 - accuracy: 0.9436 - val_loss: 0.1094 - val_accuracy: 0.9510
Epoch 3/3
648/648 [==============================] - 30s 47ms/step - loss: 0.0975 - accuracy: 0.9446 - val_loss: 0.0999 - val_accuracy: 0.9489


/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[still updating...] Accuracy: 0.9464985994397759, Precision: 0.14016565418865903, Recall: 0.0832463924150875, FScore: 0.09526316369655244
Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
masking_25 (Masking)            (None, 50)           0           input_25[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 50, 50)       72900       masking_25[0][0]                 
__________________________________________________________________________________________________
input_26 (InputLayer)           (None, 50, 14)      

...

/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[still updating...] Accuracy: 0.9438114228742345, Precision: 0.11927609562113302, Recall: 0.06208732464361825, FScore: 0.0702476013560719
Model: "model_16"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, 50)           0                                            
__________________________________________________________________________________________________
masking_31 (Masking)            (None, 50)           0           input_31[0][0]                   
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 50, 50)       72900       masking_31[0][0]                 
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, 50, 14)      

/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/envs/py36/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 648 samples, validate on 72 samples
Epoch 1/3
648/648 [==============================] - 36s 55ms/step - loss: 0.1794 - accuracy: 0.9303 - val_loss: 0.0982 - val_accuracy: 0.9599
Epoch 2/3
648/648 [==============================] - 37s 58ms/step - loss: 0.1170 - accuracy: 0.9418 - val_loss: 0.0741 - val_accuracy: 0.9592
Epoch 3/3
648/648 [==============================] - 32s 49ms/step - loss: 0.1008 - accuracy: 0.9435 - val_loss: 0.0676 - val_accuracy: 0.9585
[still updating...] Accuracy: 0.9434210526315789, Precision: 0.11161119102313831, Recall: 0.06264203472689935, FScore: 0.07010134619525694
Accuracy: 0.9434210526315789, Precision: 0.11161119102313831, Recall: 0.06264203472689935, FScore: 0.07010134619525694


/home/stud/btech/cse/2016/mukuntha.cs16/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
X_test_data = [np.asarray([a[0] for a in parsed_test_sentences]), np.asarray([a[1] for a in parsed_test_sentences])]

In [46]:
predictions_full = model.predict(X_test_data)

In [47]:
predictions_full.shape

(100, 50, 21)

In [48]:
predictions_list = []
for i, s in enumerate(test_sentences):
    output = []
    for j, w in enumerate(s[0]):
        output.append(np.argmax(predictions_full[i][j]))
    predictions_list.append(output)

In [49]:
len(predictions_list)

100

In [50]:
print(predictions_list)

[[15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15], [15, 15, 15, 15, 15, 15, 15, 15

In [51]:
Counter(sum(predictions_list, []))

Counter({15: 1881, 7: 5, 17: 15, 9: 1})